## Structure of this notebook

This notebook is structured as follows. Clicking on the link will take you directly to the cell:
* **[Imports and Definitions](#imports_and_definitions)**
    - Definition of libraries used
* **[Data Manipulation Library](#data_manipulation)**
    - Library to expose all methods relevant to manipulation of data
* **[Intelligence Library](#intelligence)**
    - Library to expose all methods relevant to implementation of intelligence algorithms
* **[Using *Archiv* dataset](#test1)**
* **[Using *Archiv/2016* dataset without *Referenz*](#test2)**
* **[Using *Archiv/2016* dataset with *Referenz*](#test3)**
* **[Combining both *Archiv* and *Archiv/2016* datasets](#test4)**

## Imports and Definitions
<a id='imports_and_definitions'></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import svm
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

import warnings
warnings.filterwarnings('ignore')

from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam

from datetime import datetime
import matplotlib.pyplot as plt

ARCHIV = '../data/Archiv/'
ARCHIV2016 = '../data/Archiv2016/'

Using TensorFlow backend.


## Data Manipulation Library
<a id='data_manipulation'></a>

In [2]:
class DataManipulation(object):
    
    def __init__(self, filename, sep_=';', dec_=','):

        self.data = pd.read_csv(filename, sep=sep_, decimal=dec_).dropna()
        self.data_array = self.data.values[:,1:].astype(dtype=float)
        
    def visualise_data(self):
        '''
            Read a given file into a pandas dataframe.
            Then, plot the various features against 
            values of frequency.
            :param: 
                None
            :returns:
                None
        '''
        self.data.plot(x='nm', y=self.data.columns[1:], kind='line', figsize=(20, 15))

    def augment_dataset(self, target):
        '''
            Augment the given dataset with the supplied
            target label value.
            :param:
                target: Target label 
            :returns:
                self.data_array: Augmented dataset
        '''
        self.data_array = np.hstack((self.data_array, \
                                     np.full((self.data_array.shape[0], 1), target)))
        return self.data_array

    def extract_features(self, no_of_features):
        '''
            Extract usable features from the dataset.
            :param:
                no_of_features: Number of features to be extracted
            :returns:
                extracted_features: Extracted features
        '''
        X = self.data_array[:, :-1]
        y = self.data_array[:, -1]

        test = SelectKBest(score_func=f_classif, k=no_of_features)
        fit = test.fit(X, y)
        extracted_features = np.hstack((fit.transform(X), y.reshape(self.data_array.shape[0], 1)))
        
        return extracted_features
    
    def align_dataset(self, other, update_other=False, debug=False):
        '''
            Align self to other dataframe, by appropriately fixing
            missing terms.
            :param:
                other: DataManipulation object to be aligned to
                update_other: Flag to update other, default False
                debug: Enable debug print statements, default False
            :returns:
                None
        '''
        self_ = self.data['nm'].values
        self_min, self_max = self_[0], self_[-1]
        
        other_ = other.data['nm'].values
        other_min, other_max = other_[0], other_[-1]
        
        min_index = self_min if self_min > other_min else other_min
        max_index = self_max if self_max < other_max else other_max
        
        self.data_array = self.data[self.data['nm'].isin(range(min_index, max_index+1, 10))]\
                                    .values[:,1:].astype(dtype=float)
        if update_other:
            other.data_array = other.data[other.data['nm'].isin(range(min_index, max_index+1, 10))]\
                                          .values[:,1:].astype(dtype=float)
        
        if debug:
            print ("Self: ", self_min, self_max, self.data_array.shape)
            print ("Other: ", other_min, other_max, other.data_array.shape)
            print ("Min/max: ", min_index, max_index)
        
    def cumulative_dataset(data_):
        '''
            Cumulate the data into one single data
            form.
            :param: 
                data_: Datasets to be accumulated
            :returns:
                dataset_: Complete dataset
        '''
        dataset_ = np.empty((0, data_[0].shape[1]))
        
        for i in data_:
            dataset_ = np.vstack((dataset_, i))
        
        return dataset_

## Intelligence Library
<a id='intelligence'></a>

In [3]:
class Intelligence(object):
    
    def __init__(self, negative_, positive_, no_test_points):
        
        np.random.shuffle(negative_)
        np.random.shuffle(positive_)
        self.train_data = np.vstack((negative_[:no_test_points, :-1], \
                                     positive_[:no_test_points, :-1]))
        self.train_labels = np.hstack((negative_[:no_test_points, -1], \
                                       positive_[:no_test_points, -1]))
        self.test_data = np.vstack((negative_[no_test_points:, :-1], \
                                    positive_[no_test_points:, :-1]))
        self.test_labels = np.hstack((negative_[no_test_points:, -1], \
                                      positive_[no_test_points:, -1]))

    def svm_(self, kernel):
        '''
            Implement SVM with user defined kernel.
            :param:
                kernel: SVM kernel to be used
            :returns:
                None
        '''
        start_ = datetime.timestamp(datetime.now())
        svm_classifier = svm.SVC(C=500, kernel=kernel)
        svm_classifier.fit(self.train_data, self.train_labels)
        stop_ = datetime.timestamp(datetime.now())
        self.time_ = stop_ - start_
        self.__print_results(model=svm_classifier, desc='SVM classifier with '+kernel+' kernel')

    def mlp_(self, l_rate=0.001, epochs=100):
        '''
            Implement MLP with 1 input layer, 3 hidden
            layer and 1 output layer.
            :param: 
                l_rate: Learning rate, default 0.001
                epochs: No of epochs, default 100
            :returns:
                None
        '''
        start_ = datetime.timestamp(datetime.now())
        input_ = Input(shape=(self.train_data.shape[1],))
        hidden_1 = Dense(self.train_data.shape[1], activation='sigmoid')(input_)
        hidden_2 = Dense(self.train_data.shape[1], activation='sigmoid')(hidden_1)
        hidden_3 = Dense(self.train_data.shape[1], activation='sigmoid')(hidden_2)
        output_ = Dense(1, activation='sigmoid')(hidden_3)
        mlp_model = Model(input_, output_)

        mlp_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=l_rate))

        weights = compute_class_weight('balanced', np.array([0,1]), self.train_labels)
        mlp_model.fit(self.train_data, self.train_labels, 
                      batch_size=10, epochs=epochs, class_weight={0:weights[0],1:weights[1]})
        predicted_labels = mlp_model.predict(self.test_data)
        predicted_labels = np.array([i > 0.5 for i in predicted_labels])
        stop_ = datetime.timestamp(datetime.now())
        self.time_ = stop_ - start_
        self.__print_results(model=predicted_labels, desc='MLP using 1-3-1 network')
        
    def __print_results(self, model, desc):
        '''
            Print the results to verify network.
            :param:
                model: Model used for classification
                desc: Description
            :returns:
                None
        '''
        model = model.predict(self.test_data) if desc[:3]=='SVM' else model
        print (desc)
        print ('-'*50)
        print ('='*5+' Train/Test Split '+'='*5+'\n {:.2f}/{:.2f} %'\
               .format(len(self.train_data)/(len(self.train_data)+len(self.test_data))*100, \
                       len(self.test_data)/(len(self.train_data)+len(self.test_data))*100))
        print ('='*5+' Confusion Matrix '+'='*5+'\n', confusion_matrix(self.test_labels, model, [1,0]))
        print ('='*5+' Precision '+'='*5+'\n {:.2f}%'.format(precision_score(self.test_labels, model, [1,0])*100))
        print ('='*5+' Recall '+'='*5+'\n {:.2f}%'.format(recall_score(self.test_labels, model, [1,0])*100))
        print ('='*5+' Execution Time '+'='*5+'\n {:.5f} sec'.format(self.time_))
        print ('-'*50+'\n')

## Using *Archiv* dataset
<a id='test1'></a>

In [4]:
## Get data into python
stoff = DataManipulation(ARCHIV+'Stoff.csv')
stoff.augment_dataset(target=0)
stoff_features = stoff.extract_features(no_of_features=6)

fleisch = DataManipulation(ARCHIV+'Fleisch.csv')
fleisch.augment_dataset(target=0)
fleisch_features = fleisch.extract_features(no_of_features=6)

holz = DataManipulation(ARCHIV+'Holz.csv')
holz.augment_dataset(target=0)
holz_features = holz.extract_features(no_of_features=6)

leder = DataManipulation(ARCHIV+'Leder.csv')
leder.augment_dataset(target=0)
leder_features = leder.extract_features(no_of_features=6)

non_skin_dataset = DataManipulation.cumulative_dataset([stoff_features, \
                                                        fleisch_features, \
                                                        holz_features, \
                                                        leder_features])

referenz = DataManipulation(ARCHIV+'Referenz-Haut_6-Klassen.csv')
skin_dataset = referenz.augment_dataset(target=1)

In [5]:
intelligence_client = Intelligence(non_skin_dataset, skin_dataset, 50)
intelligence_client.svm_(kernel='linear')
intelligence_client.svm_(kernel='rbf')

SVM classifier with linear kernel
--------------------------------------------------
===== Train/Test Split =====
 16.53/83.47 %
===== Confusion Matrix =====
 [[ 71   0]
 [ 21 413]]
===== Precision =====
 77.17%
===== Recall =====
 100.00%
===== Execution Time =====
 0.00272 sec
--------------------------------------------------

SVM classifier with rbf kernel
--------------------------------------------------
===== Train/Test Split =====
 16.53/83.47 %
===== Confusion Matrix =====
 [[ 63   8]
 [ 12 422]]
===== Precision =====
 84.00%
===== Recall =====
 88.73%
===== Execution Time =====
 0.00151 sec
--------------------------------------------------



In [6]:
intelligence_client.mlp_(l_rate=0.01, epochs=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 0.7190
Epoch 2/100
100/100 [==============================] - 0s 220us/step - loss: 0.6874
Epoch 3/100
100/100 [==============================] - 0s 262us/step - loss: 0.6830
Epoch 4/100
100/100 [==============================] - 0s 261us/step - loss: 0.6749
Epoch 5/100
100/100 [==============================] - 0s 206us/step - loss: 0.6650
Epoch 6/100
100/100 [==============================] - 0s 322us/step - loss: 0.6479
Epoch 7/100
100/100 [==============================] - 0s 197us/step - loss: 0.6216
Epoch 8/100
100/100 [==============================] - 0s 190us/step - loss: 0.5885
Epoch 9/100
100/100 [==============================] - 0s 287us/step - loss: 0.5484
Epoch 10/100
100/100 [==============================] - 0s 257us/step - loss: 0.4902
Epoch 11/100
100/100 [==============================] - 0s 272us/s

100/100 [==============================] - 0s 263us/step - loss: 0.0034
Epoch 92/100
100/100 [==============================] - 0s 285us/step - loss: 0.0033
Epoch 93/100
100/100 [==============================] - 0s 250us/step - loss: 0.0033
Epoch 94/100
100/100 [==============================] - 0s 269us/step - loss: 0.0032
Epoch 95/100
100/100 [==============================] - 0s 254us/step - loss: 0.0031
Epoch 96/100
100/100 [==============================] - 0s 213us/step - loss: 0.0031
Epoch 97/100
100/100 [==============================] - 0s 197us/step - loss: 0.0030
Epoch 98/100
100/100 [==============================] - 0s 257us/step - loss: 0.0030
Epoch 99/100
100/100 [==============================] - 0s 257us/step - loss: 0.0029
Epoch 100/100
100/100 [==============================] - 0s 193us/step - loss: 0.0028
MLP using 1-3-1 network
--------------------------------------------------
===== Train/Test Split =====
 16.53/83.47 %
===== Confusion Matrix =====
 [[ 71   0]
 [

## Using *Archiv/2016* dataset without *Referenz*
<a id='test2'></a>

In [7]:
## Get data into python
material = DataManipulation(ARCHIV2016+'2016material.csv')
material.augment_dataset(target=0)
material_features = material.extract_features(no_of_features=15)

material_fake = DataManipulation(ARCHIV2016+'2016material-fake.csv')
material_fake.augment_dataset(target=0)
material_fake_features = material_fake.extract_features(no_of_features=15)

non_skin_dataset = DataManipulation.cumulative_dataset([material_features, \
                                                        material_fake_features])

skin = DataManipulation(ARCHIV2016+'2016skin.csv')
skin.augment_dataset(target=1)
skin_dataset = skin.extract_features(no_of_features=15)

In [8]:
intelligence_client = Intelligence(non_skin_dataset, skin_dataset, 50)
intelligence_client.svm_(kernel='linear')
intelligence_client.svm_(kernel='rbf')

SVM classifier with linear kernel
--------------------------------------------------
===== Train/Test Split =====
 3.26/96.74 %
===== Confusion Matrix =====
 [[ 971    0]
 [   0 1992]]
===== Precision =====
 100.00%
===== Recall =====
 100.00%
===== Execution Time =====
 0.00055 sec
--------------------------------------------------

SVM classifier with rbf kernel
--------------------------------------------------
===== Train/Test Split =====
 3.26/96.74 %
===== Confusion Matrix =====
 [[ 952   19]
 [   0 1992]]
===== Precision =====
 100.00%
===== Recall =====
 98.04%
===== Execution Time =====
 0.00173 sec
--------------------------------------------------



In [9]:
intelligence_client.mlp_(l_rate=0.01, epochs=100)

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 0.7064
Epoch 2/100
100/100 [==============================] - 0s 209us/step - loss: 0.6552
Epoch 3/100
100/100 [==============================] - 0s 338us/step - loss: 0.5974
Epoch 4/100
100/100 [==============================] - 0s 317us/step - loss: 0.5080
Epoch 5/100
100/100 [==============================] - 0s 355us/step - loss: 0.4018
Epoch 6/100
100/100 [==============================] - 0s 257us/step - loss: 0.2790
Epoch 7/100
100/100 [==============================] - 0s 279us/step - loss: 0.1713
Epoch 8/100
100/100 [==============================] - 0s 227us/step - loss: 0.0966
Epoch 9/100
100/100 [==============================] - 0s 383us/step - loss: 0.0593
Epoch 10/100
100/100 [==============================] - 0s 271us/step - loss: 0.0390
Epoch 11/100
100/100 [==============================] - 0s 341us/step - loss: 0.0291
Epoch 12/100
100/100 [==============================] - 0s 242us/step - loss

100/100 [==============================] - 0s 182us/step - loss: 6.4875e-04
Epoch 97/100
100/100 [==============================] - 0s 223us/step - loss: 6.3769e-04
Epoch 98/100
100/100 [==============================] - 0s 233us/step - loss: 6.2691e-04
Epoch 99/100
100/100 [==============================] - 0s 271us/step - loss: 6.1668e-04
Epoch 100/100
100/100 [==============================] - 0s 250us/step - loss: 6.0636e-04
MLP using 1-3-1 network
--------------------------------------------------
===== Train/Test Split =====
 3.26/96.74 %
===== Confusion Matrix =====
 [[ 971    0]
 [   0 1992]]
===== Precision =====
 100.00%
===== Recall =====
 100.00%
===== Execution Time =====
 3.14020 sec
--------------------------------------------------



## Using *Archiv/2016* dataset with *Referenz*
<a id='test3'></a>

In [10]:
## Get data into python
material = DataManipulation(ARCHIV2016+'2016material.csv')
material_fake = DataManipulation(ARCHIV2016+'2016material-fake.csv')
skin = DataManipulation(ARCHIV2016+'2016skin.csv')
referenz = DataManipulation(ARCHIV2016+'Referenz-Haut_6-Klassen.csv')

## Align datasets
material.align_dataset(other=referenz, update_other=False)
material_fake.align_dataset(other=referenz, update_other=False)
skin.align_dataset(other=referenz, update_other=True)

## Augment datasets
material.augment_dataset(target=0)
material_fake.augment_dataset(target=0)
skin.augment_dataset(target=1)
referenz.augment_dataset(target=1)

## Extract features
material_features = material.extract_features(no_of_features=6)
material_fake_features = material_fake.extract_features(no_of_features=6)
skin_features = skin.extract_features(no_of_features=6)
referenz_features = referenz.extract_features(no_of_features=6)

## Cumulate dataset into non_skin and skin
non_skin_dataset = DataManipulation.cumulative_dataset([material_features, \
                                                        material_fake_features])

skin_dataset = DataManipulation.cumulative_dataset([skin_features, \
                                                    referenz_features])

In [11]:
intelligence_client = Intelligence(non_skin_dataset, skin_dataset, 20)
intelligence_client.svm_(kernel='linear')
intelligence_client.svm_(kernel='rbf')

SVM classifier with linear kernel
--------------------------------------------------
===== Train/Test Split =====
 10.64/89.36 %
===== Confusion Matrix =====
 [[168   0]
 [  0 168]]
===== Precision =====
 100.00%
===== Recall =====
 100.00%
===== Execution Time =====
 0.00064 sec
--------------------------------------------------

SVM classifier with rbf kernel
--------------------------------------------------
===== Train/Test Split =====
 10.64/89.36 %
===== Confusion Matrix =====
 [[168   0]
 [ 57 111]]
===== Precision =====
 74.67%
===== Recall =====
 100.00%
===== Execution Time =====
 0.00086 sec
--------------------------------------------------



In [12]:
intelligence_client.mlp_(l_rate=0.01, epochs=100)

Epoch 1/100
40/40 [==============================] - 0s 4ms/step - loss: 0.7194
Epoch 2/100
40/40 [==============================] - 0s 196us/step - loss: 0.6936
Epoch 3/100
40/40 [==============================] - 0s 181us/step - loss: 0.6940
Epoch 4/100
40/40 [==============================] - 0s 390us/step - loss: 0.6809
Epoch 5/100
40/40 [==============================] - 0s 223us/step - loss: 0.6771
Epoch 6/100
40/40 [==============================] - 0s 413us/step - loss: 0.6728
Epoch 7/100
40/40 [==============================] - 0s 287us/step - loss: 0.6658
Epoch 8/100
40/40 [==============================] - 0s 609us/step - loss: 0.6587
Epoch 9/100
40/40 [==============================] - 0s 316us/step - loss: 0.6480
Epoch 10/100
40/40 [==============================] - 0s 445us/step - loss: 0.6374
Epoch 11/100
40/40 [==============================] - 0s 315us/step - loss: 0.6229
Epoch 12/100
40/40 [==============================] - 0s 499us/step - loss: 0.6110
Epoch 13/100
40

40/40 [==============================] - 0s 379us/step - loss: 0.0106
Epoch 100/100
40/40 [==============================] - 0s 490us/step - loss: 0.0105
MLP using 1-3-1 network
--------------------------------------------------
===== Train/Test Split =====
 10.64/89.36 %
===== Confusion Matrix =====
 [[168   0]
 [  2 166]]
===== Precision =====
 98.82%
===== Recall =====
 100.00%
===== Execution Time =====
 2.59299 sec
--------------------------------------------------



## Combining both *Archiv* and *Archiv/2016* datasets
<a id='test4'></a>

In [13]:
# Archiv datase
## Get data into python
stoff = DataManipulation(ARCHIV+'Stoff.csv')
fleisch = DataManipulation(ARCHIV+'Fleisch.csv')
holz = DataManipulation(ARCHIV+'Holz.csv')
leder = DataManipulation(ARCHIV+'Leder.csv')

# Archiv 2016 dataset
## Get data into python
material = DataManipulation(ARCHIV2016+'2016material.csv')
material_fake = DataManipulation(ARCHIV2016+'2016material-fake.csv')
skin = DataManipulation(ARCHIV2016+'2016skin.csv')
referenz = DataManipulation(ARCHIV2016+'Referenz-Haut_6-Klassen.csv')

## Align datasets
material.align_dataset(other=referenz, update_other=False)
material_fake.align_dataset(other=referenz, update_other=False)
skin.align_dataset(other=referenz, update_other=True)
stoff.align_dataset(other=material, update_other=False)
fleisch.align_dataset(other=material, update_other=False)
holz.align_dataset(other=material, update_other=False)
leder.align_dataset(other=material, update_other=False)

## Augment dataset
stoff.augment_dataset(target=0)
fleisch.augment_dataset(target=0)
holz.augment_dataset(target=0)
leder.augment_dataset(target=0)
material.augment_dataset(target=0)
material_fake.augment_dataset(target=0)
skin.augment_dataset(target=1)
referenz.augment_dataset(target=1)

## Extract features
stoff_features = stoff.extract_features(no_of_features=6)
fleisch_features = fleisch.extract_features(no_of_features=6)
holz_features = holz.extract_features(no_of_features=6)
leder_features = leder.extract_features(no_of_features=6)
material_features = material.extract_features(no_of_features=6)
material_fake_features = material_fake.extract_features(no_of_features=6)
skin_features = skin.extract_features(no_of_features=6)
referenz_features = referenz.extract_features(no_of_features=6)

## Cumulate dataset into non_skin and skin
non_skin_dataset = DataManipulation.cumulative_dataset([stoff_features, \
                                                        fleisch_features, \
                                                        holz_features, \
                                                        leder_features, \
                                                        material_features, \
                                                        material_fake_features])

skin_dataset = DataManipulation.cumulative_dataset([skin_features, \
                                                    referenz_features])

In [14]:
intelligence_client = Intelligence(non_skin_dataset, skin_dataset, 100)
intelligence_client.svm_(kernel='linear')
intelligence_client.svm_(kernel='rbf')

SVM classifier with linear kernel
--------------------------------------------------
===== Train/Test Split =====
 26.60/73.40 %
===== Confusion Matrix =====
 [[ 85   3]
 [ 48 416]]
===== Precision =====
 63.91%
===== Recall =====
 96.59%
===== Execution Time =====
 0.53657 sec
--------------------------------------------------

SVM classifier with rbf kernel
--------------------------------------------------
===== Train/Test Split =====
 26.60/73.40 %
===== Confusion Matrix =====
 [[ 84   4]
 [  0 464]]
===== Precision =====
 100.00%
===== Recall =====
 95.45%
===== Execution Time =====
 0.00265 sec
--------------------------------------------------



In [15]:
intelligence_client.mlp_(l_rate=0.0075, epochs=100)

Epoch 1/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6981
Epoch 2/100
200/200 [==============================] - 0s 165us/step - loss: 0.6794
Epoch 3/100
200/200 [==============================] - 0s 240us/step - loss: 0.6657
Epoch 4/100
200/200 [==============================] - 0s 189us/step - loss: 0.6446
Epoch 5/100
200/200 [==============================] - 0s 179us/step - loss: 0.6013
Epoch 6/100
200/200 [==============================] - 0s 215us/step - loss: 0.5369
Epoch 7/100
200/200 [==============================] - 0s 194us/step - loss: 0.4667
Epoch 8/100
200/200 [==============================] - 0s 304us/step - loss: 0.3832
Epoch 9/100
200/200 [==============================] - 0s 197us/step - loss: 0.3183
Epoch 10/100
200/200 [==============================] - 0s 169us/step - loss: 0.2743
Epoch 11/100
200/200 [==============================] - 0s 214us/step - loss: 0.2249
Epoch 12/100
200/200 [==============================] - 0s 221us/step - loss

200/200 [==============================] - 0s 226us/step - loss: 0.0022
Epoch 98/100
200/200 [==============================] - 0s 291us/step - loss: 0.0022
Epoch 99/100
200/200 [==============================] - 0s 249us/step - loss: 0.0021
Epoch 100/100
200/200 [==============================] - 0s 210us/step - loss: 0.0021
MLP using 1-3-1 network
--------------------------------------------------
===== Train/Test Split =====
 26.60/73.40 %
===== Confusion Matrix =====
 [[ 88   0]
 [  2 462]]
===== Precision =====
 97.78%
===== Recall =====
 100.00%
===== Execution Time =====
 5.31044 sec
--------------------------------------------------

